In [1]:
import re
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def load_lexicon(file_path):
    lexicon = pd.read_csv(file_path, sep='\t', header=0, names=['word', 'weight'])
    lexicon['weight'] = pd.to_numeric(lexicon['weight'], errors='coerce')
    lexicon['length'] = lexicon['word'].apply(lambda x: len(x.split()))
    return lexicon.sort_values(by='length', ascending=False).reset_index(drop=True)

positive_lexicon = load_lexicon('positive.tsv')
negative_lexicon = load_lexicon('negative.tsv')

def calculate_sentiment(text, positive_lexicon, negative_lexicon):
    text = preprocess_text(text)
    sentiment_score = 0
    words = text.split()
    
    i = 0
    while i < len(words):
        matched = False
        for j in range(len(words), i, -1):
            phrase = ' '.join(words[i:j])
            
            pos_match = positive_lexicon[positive_lexicon['word'] == phrase]
            if not pos_match.empty:
                sentiment_score += pos_match.iloc[0]['weight']
                # Skip the matched words
                i = j - 1
                matched = True
                break
            
            neg_match = negative_lexicon[negative_lexicon['word'] == phrase]
            if not neg_match.empty:
                sentiment_score += neg_match.iloc[0]['weight']
                # Skip the matched words
                i = j - 1
                matched = True
                break
        if not matched:
            i += 1
        else:
            i += 1
    return sentiment_score

# 1. Load the same dataset
ds = load_dataset("rizr09/fin-dataset")
df = pd.DataFrame(ds['train'])

# 2. Perform the same data splitting
train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# Rename columns
train_df = train_df.rename(columns={"text_clean": "text", "label": "label"})
eval_df = eval_df.rename(columns={"text_clean": "text", "label": "label"})

# Keep only necessary columns
train_df = train_df[['text', 'label']]
eval_df = eval_df[['text', 'label']]

# Reset index
train_df = train_df.reset_index(drop=True)
eval_df = eval_df.reset_index(drop=True)

# 3. Apply calculate_sentiment to the evaluation dataset
eval_df['Lexicon Sentiment Score'] = eval_df['text'].apply(lambda x: calculate_sentiment(x, positive_lexicon, negative_lexicon))

# 4. Map sentiment scores to labels
def sentiment_score_to_label(score):
    if score < 0:
        return 0  # negative
    elif score == 0:
        return 1  # neutral
    else:
        return 2  # positive

eval_df['Predicted Label'] = eval_df['Lexicon Sentiment Score'].apply(sentiment_score_to_label)

# 5. Generate the classification report
true_labels = eval_df['label']
predicted_labels = eval_df['Predicted Label']

# Define label mapping
label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
target_names = [label_mapping[i] for i in sorted(label_mapping.keys())]

report = classification_report(true_labels, predicted_labels, target_names=target_names)

print("Classification Report:")
print(report)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

c:\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP VICTUS 16\.cache\huggingface\hub\datasets--rizr09--fin-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data.csv:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5842 [00:00<?, ? examples/s]

Classification Report:
              precision    recall  f1-score   support

    negative       0.18      0.54      0.27       175
     neutral       0.42      0.06      0.10       622
    positive       0.34      0.51      0.41       372

    accuracy                           0.27      1169
   macro avg       0.31      0.37      0.26      1169
weighted avg       0.36      0.27      0.22      1169

